# DEADMAN COMMAND CENTER
## Mission Control | Task Management | Intelligence Operations

---

**Classification:** OPERATIONAL  
**Version:** 1.0.0  
**Last Updated:** 2026-01-13  
**Status:** ACTIVE

---

### System Architecture
```
+==================================================================================+
|                        DEADMAN INTELLIGENCE COMMAND                              |
|                          "The Unseen. The Unstoppable."                         |
+==================================================================================+
|  [QUEUE]  |  [ACTIVE]  |  [ERRORS]  |  [COMPLETE]  |  [RESEARCH]  |  [BUILDS]  |
+==================================================================================+
```

In [ ]:
# ============================================================================
# DEADMAN COMMAND CENTER - Core Configuration
# ============================================================================

import json
import os
from datetime import datetime
from pathlib import Path
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Optional, Any
from enum import Enum
import hashlib

# Base paths
BASE_DIR = Path("G:/DeadManUltimateScraper")
DATA_DIR = BASE_DIR / "data"
TASKS_DIR = DATA_DIR / "tasks"
TASKS_DIR.mkdir(parents=True, exist_ok=True)

print(f"DEADMAN COMMAND CENTER INITIALIZED")
print(f"Base: {BASE_DIR}")
print(f"Tasks: {TASKS_DIR}")

---
## 01 | TASK CLASSIFICATION SYSTEM
---

In [ ]:
# ============================================================================
# TASK STATUS & PRIORITY ENUMS
# ============================================================================

class TaskStatus(Enum):
    """Task lifecycle states."""
    QUEUED = "QUEUED"           # Waiting to start
    ACTIVE = "ACTIVE"           # Currently in progress
    BLOCKED = "BLOCKED"         # Waiting on dependency
    ERROR = "ERROR"             # Failed - needs attention
    RETRY = "RETRY"             # Scheduled for retry
    REVIEW = "REVIEW"           # Needs human review
    COMPLETE = "COMPLETE"       # Successfully finished
    ARCHIVED = "ARCHIVED"       # Historical record

class TaskPriority(Enum):
    """Task priority levels (NASA-style)."""
    P0_CRITICAL = 0    # Mission-critical, immediate
    P1_HIGH = 1        # High priority, same day
    P2_MEDIUM = 2      # Standard priority
    P3_LOW = 3         # Background task
    P4_BACKLOG = 4     # When resources available

class TaskCategory(Enum):
    """Task categories for organization."""
    # Core Operations
    SCRAPE = "SCRAPE"           # Web scraping tasks
    RESEARCH = "RESEARCH"       # Research & analysis
    BUILD = "BUILD"             # Development/coding
    REPAIR = "REPAIR"           # Bug fixes
    
    # Data Operations
    EXTRACT = "EXTRACT"         # Data extraction
    TRANSFORM = "TRANSFORM"     # Data processing
    LOAD = "LOAD"               # Data storage
    
    # Intelligence
    INTEL = "INTEL"             # Intelligence gathering
    ANALYSIS = "ANALYSIS"       # Data analysis
    REPORT = "REPORT"           # Report generation
    
    # Infrastructure
    INFRA = "INFRA"             # Infrastructure
    SECURITY = "SECURITY"       # Security tasks
    OPTIMIZE = "OPTIMIZE"       # Performance
    
    # Content
    YOUTUBE = "YOUTUBE"         # YouTube content
    TIKTOK = "TIKTOK"           # TikTok content
    INSTAGRAM = "INSTAGRAM"     # Instagram content
    X_TWITTER = "X_TWITTER"     # X/Twitter content
    
    # Meta
    IDEA = "IDEA"               # Ideas/concepts
    FINALIZED = "FINALIZED"     # Production-ready

print("Task Classification System Loaded")
print(f"  Statuses: {len(TaskStatus)}")
print(f"  Priorities: {len(TaskPriority)}")
print(f"  Categories: {len(TaskCategory)}")

In [ ]:
# ============================================================================
# TASK DATA MODEL
# ============================================================================

@dataclass
class Task:
    """Individual task record."""
    # Identity
    id: str                           # Unique hash ID
    title: str                        # Short title
    description: str                  # Detailed description
    
    # Classification
    category: TaskCategory
    priority: TaskPriority
    status: TaskStatus
    
    # Metadata
    created_at: str                   # ISO timestamp
    updated_at: str                   # Last update
    due_date: Optional[str] = None    # Target completion
    
    # Tracking
    assigned_to: str = "SYSTEM"       # Owner
    tags: List[str] = field(default_factory=list)
    dependencies: List[str] = field(default_factory=list)  # Task IDs
    
    # Progress
    progress_pct: int = 0             # 0-100
    attempts: int = 0                 # Retry count
    max_attempts: int = 3             # Max retries
    
    # Results
    result: Optional[Dict] = None     # Output data
    error: Optional[str] = None       # Error message
    logs: List[str] = field(default_factory=list)
    
    # Resources
    urls: List[str] = field(default_factory=list)     # Related URLs
    files: List[str] = field(default_factory=list)    # Related files
    tokens_used: int = 0                              # LLM tokens
    cost_usd: float = 0.0                             # API costs
    
    def __post_init__(self):
        """Generate ID if not provided."""
        if not self.id:
            content = f"{self.title}{self.created_at}"
            self.id = hashlib.md5(content.encode()).hexdigest()[:12]
    
    def log(self, message: str):
        """Add timestamped log entry."""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.logs.append(f"[{timestamp}] {message}")
        self.updated_at = datetime.now().isoformat()
    
    def to_dict(self) -> Dict:
        """Convert to dictionary for JSON serialization."""
        d = asdict(self)
        d['category'] = self.category.value
        d['priority'] = self.priority.value
        d['status'] = self.status.value
        return d
    
    @classmethod
    def from_dict(cls, data: Dict) -> 'Task':
        """Create from dictionary."""
        data['category'] = TaskCategory(data['category'])
        data['priority'] = TaskPriority(data['priority'])
        data['status'] = TaskStatus(data['status'])
        return cls(**data)

print("Task Data Model Loaded")

---
## 02 | TASK MANAGEMENT ENGINE
---

In [ ]:
# ============================================================================
# TASK MANAGER - Core Operations
# ============================================================================

class TaskManager:
    """Central task management system."""
    
    def __init__(self, tasks_dir: Path = TASKS_DIR):
        self.tasks_dir = tasks_dir
        self.tasks_file = tasks_dir / "master_tasks.json"
        self.tasks: Dict[str, Task] = {}
        self._load_tasks()
    
    def _load_tasks(self):
        """Load tasks from disk."""
        if self.tasks_file.exists():
            with open(self.tasks_file, 'r') as f:
                data = json.load(f)
                for task_data in data.get('tasks', []):
                    try:
                        task = Task.from_dict(task_data)
                        self.tasks[task.id] = task
                    except Exception as e:
                        print(f"Error loading task: {e}")
    
    def _save_tasks(self):
        """Persist tasks to disk."""
        data = {
            'version': '1.0.0',
            'updated': datetime.now().isoformat(),
            'count': len(self.tasks),
            'tasks': [t.to_dict() for t in self.tasks.values()]
        }
        with open(self.tasks_file, 'w') as f:
            json.dump(data, f, indent=2)
    
    def create_task(
        self,
        title: str,
        description: str,
        category: TaskCategory,
        priority: TaskPriority = TaskPriority.P2_MEDIUM,
        **kwargs
    ) -> Task:
        """Create and register a new task."""
        now = datetime.now().isoformat()
        task = Task(
            id="",
            title=title,
            description=description,
            category=category,
            priority=priority,
            status=TaskStatus.QUEUED,
            created_at=now,
            updated_at=now,
            **kwargs
        )
        self.tasks[task.id] = task
        task.log(f"Task created: {title}")
        self._save_tasks()
        return task
    
    def get_task(self, task_id: str) -> Optional[Task]:
        """Get task by ID."""
        return self.tasks.get(task_id)
    
    def update_status(self, task_id: str, status: TaskStatus, message: str = ""):
        """Update task status."""
        task = self.tasks.get(task_id)
        if task:
            task.status = status
            task.log(f"Status -> {status.value}: {message}")
            self._save_tasks()
    
    def set_error(self, task_id: str, error: str):
        """Mark task as error."""
        task = self.tasks.get(task_id)
        if task:
            task.status = TaskStatus.ERROR
            task.error = error
            task.attempts += 1
            task.log(f"ERROR: {error}")
            self._save_tasks()
    
    def complete_task(self, task_id: str, result: Dict = None):
        """Mark task as complete."""
        task = self.tasks.get(task_id)
        if task:
            task.status = TaskStatus.COMPLETE
            task.progress_pct = 100
            task.result = result
            task.log("Task completed successfully")
            self._save_tasks()
    
    def get_by_status(self, status: TaskStatus) -> List[Task]:
        """Get all tasks with given status."""
        return [t for t in self.tasks.values() if t.status == status]
    
    def get_by_category(self, category: TaskCategory) -> List[Task]:
        """Get all tasks in category."""
        return [t for t in self.tasks.values() if t.category == category]
    
    def get_errors(self) -> List[Task]:
        """Get all error tasks."""
        return self.get_by_status(TaskStatus.ERROR)
    
    def get_active(self) -> List[Task]:
        """Get all active tasks."""
        return self.get_by_status(TaskStatus.ACTIVE)
    
    def get_queue(self) -> List[Task]:
        """Get queued tasks sorted by priority."""
        queued = self.get_by_status(TaskStatus.QUEUED)
        return sorted(queued, key=lambda t: t.priority.value)

# Initialize global task manager
TM = TaskManager()
print(f"Task Manager Initialized")
print(f"  Loaded: {len(TM.tasks)} tasks")

---
## 03 | CURRENT TASK QUEUE
---

In [ ]:
# ============================================================================
# INITIALIZE TASK QUEUE - Reddit TOR Scrapes
# ============================================================================

# r/TOR URLs to scrape
tor_urls = [
    "https://www.reddit.com/r/TOR/",
    "https://www.reddit.com/r/TOR/search/?q=reverse+engineer",
    "https://www.reddit.com/r/TOR/search/?q=politics",
    "https://www.reddit.com/r/TOR/search/?q=blackmail",
    "https://www.reddit.com/r/TOR/search/?q=earn",
    "https://www.reddit.com/r/TOR/search/?q=income",
]

# Create scrape tasks
for url in tor_urls:
    query = url.split('q=')[-1].replace('+', ' ') if 'q=' in url else 'main'
    TM.create_task(
        title=f"Scrape r/TOR: {query}",
        description=f"Scrape Reddit r/TOR for: {query}",
        category=TaskCategory.SCRAPE,
        priority=TaskPriority.P1_HIGH,
        tags=["reddit", "tor", "scrape"],
        urls=[url]
    )

print(f"Created {len(tor_urls)} r/TOR scrape tasks")

In [ ]:
# ============================================================================
# ADD PREVIOUSLY FAILED TASKS FOR RETRY
# ============================================================================

# Failed scrapes that need retry
failed_scrapes = [
    # GitHub content extraction errors
    ("GitHub: kayba-ai/agentic-context-engine", "https://github.com/kayba-ai/agentic-context-engine"),
    ("GitHub: 0xrdan/claude-router", "https://github.com/0xrdan/claude-router"),
    ("GitHub: CodebuffAI/codebuff", "https://github.com/CodebuffAI/codebuff"),
    ("GitHub: oakenai/mcp-edit-file-lines", "https://github.com/oakenai/mcp-edit-file-lines"),
    
    # Claude docs that failed
    ("Claude: Prompt Caching Deep Dive", "https://platform.claude.com/docs/en/build-with-claude/prompt-caching"),
    
    # Reddit workflow posts (rate limited)
    ("Reddit: Building Complex Apps with Claude", "https://www.reddit.com/r/ClaudeAI/comments/1hb9f70/my_process_for_building_complex_apps_using_claude/"),
    ("Reddit: The Ralph Wiggum Loop", "https://www.reddit.com/r/ClaudeCode/comments/1q9qjk4/the_ralphwiggum_loop/"),
]

for title, url in failed_scrapes:
    TM.create_task(
        title=f"RETRY: {title}",
        description=f"Retry failed scrape: {url}",
        category=TaskCategory.REPAIR,
        priority=TaskPriority.P1_HIGH,
        tags=["retry", "scrape", "error"],
        urls=[url]
    )

print(f"Created {len(failed_scrapes)} retry tasks")

In [ ]:
# ============================================================================
# ADD RESEARCH TASKS
# ============================================================================

research_tasks = [
    ("Extract ACE prompts_v2_1.py templates", "Extract all prompt templates from ACE framework", ["ace", "prompts", "templates"]),
    ("Analyze Cursor speculative edits", "Deep dive into speculative edit implementation", ["cursor", "speculative", "ai"]),
    ("Document Shadow Workspace pattern", "Document shadow workspace implementation for background execution", ["cursor", "shadow", "background"]),
    ("Research BM25 + Embeddings hybrid search", "Implement hybrid search for codebase", ["search", "bm25", "embeddings"]),
    ("Catalog all .onion search engines", "Create comprehensive onion search engine index", ["onion", "search", "tor"]),
]

for title, desc, tags in research_tasks:
    TM.create_task(
        title=title,
        description=desc,
        category=TaskCategory.RESEARCH,
        priority=TaskPriority.P2_MEDIUM,
        tags=tags
    )

print(f"Created {len(research_tasks)} research tasks")

In [ ]:
# ============================================================================
# ADD BUILD TASKS
# ============================================================================

build_tasks = [
    ("Implement Speculative Edits Engine", "Port Cursor-style speculative edits to scraper", TaskPriority.P2_MEDIUM),
    ("Build Shadow Workspace Module", "Background code execution with linter feedback", TaskPriority.P2_MEDIUM),
    ("Create Hybrid Search Pipeline", "BM25 + embeddings + reranking", TaskPriority.P2_MEDIUM),
    ("Implement FREE LLM Router", "Route to Mistral/Groq/Cerebras based on task", TaskPriority.P1_HIGH),
    ("Build Obsidian Vault Integration", "Direct output to Obsidian vault", TaskPriority.P3_LOW),
    ("Create CLI Interface", "deadman command line tool", TaskPriority.P2_MEDIUM),
]

for title, desc, priority in build_tasks:
    TM.create_task(
        title=title,
        description=desc,
        category=TaskCategory.BUILD,
        priority=priority,
        tags=["build", "development"]
    )

print(f"Created {len(build_tasks)} build tasks")

In [ ]:
# ============================================================================
# ADD CONTENT TASKS (Social Media)
# ============================================================================

content_tasks = [
    ("YouTube: Scraper Demo Video", TaskCategory.YOUTUBE, ["demo", "tutorial"]),
    ("YouTube: TOR Bypass Techniques", TaskCategory.YOUTUBE, ["tor", "bypass", "security"]),
    ("TikTok: Quick Tips Series", TaskCategory.TIKTOK, ["tips", "short-form"]),
    ("Instagram: Infographics", TaskCategory.INSTAGRAM, ["infographic", "visual"]),
    ("X: Thread on AI Scraping", TaskCategory.X_TWITTER, ["thread", "ai", "scraping"]),
]

for title, category, tags in content_tasks:
    TM.create_task(
        title=title,
        description=f"Content creation: {title}",
        category=category,
        priority=TaskPriority.P3_LOW,
        tags=tags
    )

print(f"Created {len(content_tasks)} content tasks")

---
## 04 | DASHBOARD & REPORTING
---

In [ ]:
# ============================================================================
# TASK DASHBOARD
# ============================================================================

def print_dashboard():
    """Print task management dashboard."""
    print("\n" + "=" * 80)
    print("                    DEADMAN COMMAND CENTER - DASHBOARD")
    print("=" * 80)
    print(f"  Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)
    
    # Status counts
    print("\n  STATUS OVERVIEW")
    print("  " + "-" * 40)
    for status in TaskStatus:
        count = len(TM.get_by_status(status))
        bar = "#" * min(count, 20)
        print(f"  {status.value:12} | {count:4} | {bar}")
    
    # Category counts
    print("\n  CATEGORY BREAKDOWN")
    print("  " + "-" * 40)
    for category in TaskCategory:
        count = len(TM.get_by_category(category))
        if count > 0:
            print(f"  {category.value:12} | {count:4}")
    
    # Priority queue
    print("\n  PRIORITY QUEUE (Top 10)")
    print("  " + "-" * 60)
    queue = TM.get_queue()[:10]
    for i, task in enumerate(queue, 1):
        print(f"  {i:2}. [P{task.priority.value}] {task.title[:45]}")
    
    # Errors
    errors = TM.get_errors()
    if errors:
        print(f"\n  ERRORS ({len(errors)})")
        print("  " + "-" * 60)
        for task in errors[:5]:
            print(f"  - {task.title[:50]}")
            if task.error:
                print(f"    Error: {task.error[:40]}...")
    
    print("\n" + "=" * 80)

print_dashboard()

In [ ]:
# ============================================================================
# EXPORT FUNCTIONS
# ============================================================================

def export_to_markdown():
    """Export task list to markdown."""
    md_file = TASKS_DIR / "TASK_LIST.md"
    
    lines = [
        "# DEADMAN COMMAND CENTER - Task List",
        f"\n*Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*",
        f"\n**Total Tasks:** {len(TM.tasks)}",
        "\n---\n",
    ]
    
    # Group by status
    for status in [TaskStatus.ACTIVE, TaskStatus.QUEUED, TaskStatus.ERROR, TaskStatus.COMPLETE]:
        tasks = TM.get_by_status(status)
        if tasks:
            lines.append(f"\n## {status.value} ({len(tasks)})\n")
            for task in tasks:
                checkbox = "[x]" if status == TaskStatus.COMPLETE else "[ ]"
                lines.append(f"- {checkbox} **{task.title}** [P{task.priority.value}]")
                lines.append(f"  - Category: {task.category.value}")
                if task.urls:
                    lines.append(f"  - URL: {task.urls[0]}")
    
    with open(md_file, 'w') as f:
        f.write('\n'.join(lines))
    
    print(f"Exported to: {md_file}")

export_to_markdown()

---
## 05 | QUICK ACTIONS
---

In [ ]:
# ============================================================================
# QUICK ACTION HELPERS
# ============================================================================

def add_scrape_task(url: str, title: str = None, priority: int = 2):
    """Quick add a scrape task."""
    title = title or f"Scrape: {url[:40]}"
    return TM.create_task(
        title=title,
        description=f"Scrape URL: {url}",
        category=TaskCategory.SCRAPE,
        priority=TaskPriority(priority),
        urls=[url]
    )

def add_research_task(topic: str, description: str = "", priority: int = 2):
    """Quick add a research task."""
    return TM.create_task(
        title=f"Research: {topic}",
        description=description or f"Research topic: {topic}",
        category=TaskCategory.RESEARCH,
        priority=TaskPriority(priority)
    )

def add_build_task(feature: str, description: str = "", priority: int = 2):
    """Quick add a build task."""
    return TM.create_task(
        title=f"Build: {feature}",
        description=description or f"Implement: {feature}",
        category=TaskCategory.BUILD,
        priority=TaskPriority(priority)
    )

def add_error_task(error_desc: str, url: str = None, priority: int = 1):
    """Quick add an error/repair task."""
    return TM.create_task(
        title=f"FIX: {error_desc[:40]}",
        description=error_desc,
        category=TaskCategory.REPAIR,
        priority=TaskPriority(priority),
        urls=[url] if url else []
    )

print("Quick action helpers loaded:")
print("  - add_scrape_task(url, title, priority)")
print("  - add_research_task(topic, description, priority)")
print("  - add_build_task(feature, description, priority)")
print("  - add_error_task(error_desc, url, priority)")

In [ ]:
# ============================================================================
# BATCH OPERATIONS
# ============================================================================

def add_urls_batch(urls: list, category: str = "SCRAPE", priority: int = 2):
    """Add multiple URLs as tasks."""
    cat = TaskCategory[category.upper()]
    created = []
    for url in urls:
        task = TM.create_task(
            title=f"Scrape: {url.split('/')[-1][:30] or url[:30]}",
            description=f"Scrape: {url}",
            category=cat,
            priority=TaskPriority(priority),
            urls=[url]
        )
        created.append(task.id)
    print(f"Created {len(created)} tasks from URL batch")
    return created

def retry_all_errors():
    """Mark all error tasks for retry."""
    errors = TM.get_errors()
    for task in errors:
        if task.attempts < task.max_attempts:
            TM.update_status(task.id, TaskStatus.RETRY, "Scheduled for retry")
    print(f"Scheduled {len(errors)} tasks for retry")

def archive_completed():
    """Archive all completed tasks."""
    completed = TM.get_by_status(TaskStatus.COMPLETE)
    for task in completed:
        TM.update_status(task.id, TaskStatus.ARCHIVED, "Auto-archived")
    print(f"Archived {len(completed)} completed tasks")

print("Batch operations loaded:")
print("  - add_urls_batch(urls, category, priority)")
print("  - retry_all_errors()")
print("  - archive_completed()")

---
## 06 | SYSTEM STATISTICS
---

In [ ]:
# ============================================================================
# STATISTICS TRACKER
# ============================================================================

@dataclass
class SystemStats:
    """Track system-wide statistics."""
    total_tasks_created: int = 0
    total_tasks_completed: int = 0
    total_errors: int = 0
    total_retries: int = 0
    total_urls_scraped: int = 0
    total_tokens_used: int = 0
    total_cost_usd: float = 0.0
    session_start: str = field(default_factory=lambda: datetime.now().isoformat())
    
    def to_dict(self) -> Dict:
        return asdict(self)
    
    def report(self):
        """Print statistics report."""
        print("\n" + "=" * 50)
        print("         SYSTEM STATISTICS")
        print("=" * 50)
        print(f"  Session Start: {self.session_start}")
        print(f"  Tasks Created: {self.total_tasks_created}")
        print(f"  Tasks Completed: {self.total_tasks_completed}")
        print(f"  Errors: {self.total_errors}")
        print(f"  Retries: {self.total_retries}")
        print(f"  URLs Scraped: {self.total_urls_scraped}")
        print(f"  Tokens Used: {self.total_tokens_used:,}")
        print(f"  Total Cost: ${self.total_cost_usd:.4f}")
        print("=" * 50)

# Initialize stats
STATS = SystemStats()
STATS.total_tasks_created = len(TM.tasks)
STATS.report()

---
## 07 | FINAL DASHBOARD
---

In [ ]:
# ============================================================================
# FINAL DASHBOARD OUTPUT
# ============================================================================

print_dashboard()

print("\n" + "=" * 80)
print("                    DEADMAN COMMAND CENTER - READY")
print("=" * 80)
print("\n  Available Commands:")
print("  ------------------")
print("  TM.create_task(...)     - Create new task")
print("  TM.get_queue()          - View priority queue")
print("  TM.get_errors()         - View error tasks")
print("  TM.update_status(...)   - Update task status")
print("  TM.complete_task(...)   - Mark task complete")
print("  ")
print("  Quick Actions:")
print("  add_scrape_task(url)    - Add URL to scrape")
print("  add_research_task(...)  - Add research task")
print("  add_build_task(...)     - Add build task")
print("  add_urls_batch(urls)    - Batch add URLs")
print("  retry_all_errors()      - Retry failed tasks")
print("  print_dashboard()       - Refresh dashboard")
print("  export_to_markdown()    - Export to MD")
print("\n" + "=" * 80)
print("                        THE UNSEEN. THE UNSTOPPABLE.")
print("=" * 80)